In [320]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import math
import time

#Ask user which Amazon product they want to scrape. 
amazon = input('ASIN? ')
driver = webdriver.Chrome('/usr/local/Caskroom/chromedriver/93.0.4577.63/chromedriver')
driver.get("https://www.amazon.com/product-reviews/{}".format(amazon))

#code to get review and put it into a dataframe
def get_review():
    reviews = driver.find_elements_by_class_name('review-text')
    review_list = []
    for review in reviews:
        z = review.text.lower().replace('\n',' ')
        review_list.append(z)
    x = pd.DataFrame(review_list)
    return x

#this code extract reviews stars
def get_stars():
    stars = driver.find_elements_by_class_name("review-rating")[2:]
    star_reviews = []
    for star in stars:
        star_reviews.append(star.get_attribute("innerText"))
    return star_reviews

#clicking next
def click_next():
    nx = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div/span/div/ul/li[2]/a')
    nx.click()
    
#find how many reviews a product has
#get rid of the comma in a number example: 1,000
#divide number of reviews by 10 - each page has 10 reviews
#round up to get the number of pages we have to scrape  
def num_pages():
    number_reviews_string = driver.find_element_by_xpath('//*[@id="filter-info-section"]/div/span').text
    num = number_reviews_string.split('|')[1].replace(',','')
    number_reviews = [int(s) for s in str.split(num) if s.isdigit()]
    return number_reviews[0]

ASIN? B0939MYHMT


In [321]:
#assign number of pages to num_pages: how many pages of reviews do the item has?
num_pages = num_pages()

review_table = []
star_table = []

try:
    for i in range(1, num_pages - 1):
        m = get_review()
        n = get_stars()
        review_table.append(m)
        star_table.append(n)
        click_next()
        time.sleep(3)
except NoSuchElementException:
    print("Completed!")

Completed!


In [322]:
x = pd.concat(review_table, ignore_index=True)

star_table = pd.DataFrame(star_table)
stars = star_table.to_numpy().flatten().tolist()

x = pd.DataFrame(x)
stars = pd.DataFrame(stars)

df = pd.merge(stars, x, left_index=True, right_index=True, how='outer')
df.columns = ['Stars','Reviews']
df.head()

,Stars,Reviews
0,4.0 out of 5 stars,"performs as advertised. highly recommend. my feedback below. effective device - light weight - secure - comfortable (after initial 5 minutes of running and proper adjustment) - compact - pocket for keys and id could be more effective if... - zipper makes noise while running. noticeable if out running without headphones. must tuck zipper pull inside zipper pocket as a fix - iphone 12 plus fits well. hard to get in and out on hot (texas) day due to sticking of plastic and moisture - vest has a snug fit on a very hot day gets occasionally uncomfortable. not a concern in spring, fall, or winter months. only on hot and humid days when you are running 10 mile + runs - removing after a long, hard run is doable but a front clip would be advantageous"
1,3.0 out of 5 stars,good concept and hope it will last. however i’m disappointed that the space for the phone will not allow a phone with ottherbox covering to fit. so i’m gonna have to remove the cover every time i want to use this which as you may know is a real pain in the neck. i should have asked this question prior to purchase.
2,5.0 out of 5 stars,"i never realized how great and free it feels to run without your phone in your hand until i tried it. of course you don't want to run with your phone banging against your thigh while it's in your pocket, so this vest solves the problem. it keeps your phone nice and snug against your chest. it has a single button which secures the top of the phone pouch to the vest, so when you need to check your phone, it's really easy to undo the button, check your phone, and button it back up. the phone pouch fits all phone sizes, and the one in the photos is an iphone xs max which it fits perfectly. the vest also has two smaller pockets on the shoulders which i use for my car key so i don't risk it falling out of my shorts pocket during a run. i can't recommend this product enough!"
3,1.0 out of 5 stars,product came fast and packaged nicely! super lightweight bag and trendy design. however the item it self smells like the worst material you could use for a running product. i don’t know how it could still smell while at amazon. as well as transport. leads me to think it will always smell like plastic\rubber blend. will report back-
4,2.0 out of 5 stars,yeah this sucks having to squeeze your iphone 11 into the phone pocket and ripping it on initial unboxing. has a good feel but not much room for the phone and reflective isn’t the best.


In [323]:
#export the file in csv 
df.to_csv('rev.csv', index=False)